## Sending my training plots

In [2]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [3]:
last_update_id = None

def send_telegram_message(message):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    chat_id = '5255840420'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response = requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}")

def send_telegram_image(image_path):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    chat_id = '5255840420'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    with open(image_path, 'rb') as image_file:
        files = {'photo': image_file}
        data = {'chat_id': chat_id}
        response = requests.post(url, files=files, data=data)
    print(f"Sent image: {image_path}, Response: {response.json()}")

def get_updates(offset=None):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    params = {'offset': offset, 'timeout': 30}
    response = requests.get(url, params=params).json()
    return response.get('result', [])

def clear_previous_messages():
    global last_update_id
    updates = get_updates()
    if updates:
        last_update_id = updates[-1]['update_id'] + 1
    print(f"Cleared previous messages. Last update ID: {last_update_id}")

def request_next_command():
    global last_update_id
    send_telegram_message("Please enter the next command (rerun, stop):")
    
    start_time = time.time()
    timeout = 300  # 5 minutes timeout

    while time.time() - start_time < timeout:
        updates = get_updates(last_update_id)
        for update in updates:
            last_update_id = update['update_id'] + 1
            if 'message' in update and 'text' in update['message']:
                command = update['message']['text'].lower()
                if command in ['rerun', 'stop']:return command
                else:send_telegram_message("Invalid command. Please enter 'rerun' to rerun or 'stop' to stop training.")
        time.sleep(1)
    send_telegram_message("No valid command received within 5 minutes. Stopping the program.")
    return "stop"

In [4]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),nn.ReLU(),nn.Linear(512, 10))
    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(num_epochs, train_loader, model, criterion, optimizer):
    model.train()
    epoch_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        epoch_loss /= len(train_loader)
        epoch_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
    return epoch_losses

In [5]:
clear_previous_messages()

while True:
    send_telegram_message("Model training has started.")
    epoch_losses = train_model(2, train_loader, model, criterion, optimizer)
    final_loss = epoch_losses[-1]
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.close()

    send_telegram_image('training_loss_plot.png')

    command = request_next_command()
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break
    elif command == "rerun":
        send_telegram_message("Rerunning the training.")
        continue

Cleared previous messages. Last update ID: 265526397
Sent message: Model training has started., Response: {'ok': True, 'result': {'message_id': 218, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721909349, 'text': 'Model training has started.'}}
Epoch 1, Loss: 0.3111
Epoch 2, Loss: 0.1430
Sent message: Training complete. Final loss: 0.1430, Response: {'ok': True, 'result': {'message_id': 219, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721909473, 'text': 'Training complete. Final loss: 0.1430'}}
Sent image: training_loss_plot.png, Response: {'ok': True, 'result': {'message_id': 220, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, '

---------------------------

# hyperparameters

In [ ]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

last_update_id = None

# ... (keep the existing send_telegram_message, send_telegram_image, get_updates, and clear_previous_messages functions)

def request_user_input(prompt):
    global last_update_id
    send_telegram_message(prompt)
    
    start_time = time.time()
    timeout = 300  # 5 minutes timeout

    while time.time() - start_time < timeout:
        updates = get_updates(last_update_id)
        for update in updates:
            last_update_id = update['update_id'] + 1
            if 'message' in update and 'text' in update['message']:
                return update['message']['text'].lower()
        time.sleep(1)
    
    send_telegram_message("No input received within 5 minutes. Using default value.")
    return None

def get_hyperparameters():
    choice = request_user_input("Do you want to input hyperparameters? (yes/no)")
    
    if choice == 'yes':
        learning_rate = float(request_user_input("Enter learning rate (e.g., 0.001):") or 0.001)
        batch_size = int(request_user_input("Enter batch size (e.g., 64):") or 64)
        num_epochs = int(request_user_input("Enter number of epochs (e.g., 2):") or 2)
        hidden_size = int(request_user_input("Enter hidden layer size (e.g., 512):") or 512)
    else:
        learning_rate = 0.001
        batch_size = 64
        num_epochs = 2
        hidden_size = 512
    
    send_telegram_message(f"Using hyperparameters: learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}, hidden_size={hidden_size}")
    return learning_rate, batch_size, num_epochs, hidden_size

class SimpleNet(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

def train_model(num_epochs, train_loader, model, criterion, optimizer):
    model.train()
    epoch_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        epoch_loss /= len(train_loader)
        epoch_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
        send_telegram_message(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
    return epoch_losses

# Clear previous messages before starting the main loop
clear_previous_messages()

while True:
    learning_rate, batch_size, num_epochs, hidden_size = get_hyperparameters()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    model = SimpleNet(hidden_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    send_telegram_message("Model training has started.")
    epoch_losses = train_model(num_epochs, train_loader, model, criterion, optimizer)
    final_loss = epoch_losses[-1]
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.close()

    send_telegram_image('training_loss_plot.png')

    command = request_user_input("Enter 'rerun' to train again with new parameters, or 'stop' to end the program:")
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break
    elif command == "rerun":
        send_telegram_message("Rerunning the training with new parameters.")
        continue
    else:
        send_telegram_message("Invalid command. Stopping the program.")
        break

# github

In [1]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import git
from git import Repo

last_update_id = None

def send_telegram_message(message):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    chat_id = '5255840420'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response = requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}")

def send_telegram_image(image_path):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    chat_id = '5255840420'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    with open(image_path, 'rb') as image_file:
        files = {'photo': image_file}
        data = {'chat_id': chat_id}
        response = requests.post(url, files=files, data=data)
    print(f"Sent image: {image_path}, Response: {response.json()}")

def get_updates(offset=None):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'  #creds
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    params = {'offset': offset, 'timeout': 30}
    response = requests.get(url, params=params).json()
    return response.get('result', [])

def clear_previous_messages():
    global last_update_id
    updates = get_updates()
    if updates:
        last_update_id = updates[-1]['update_id'] + 1
    print(f"Cleared previous messages. Last update ID: {last_update_id}")

def request_user_input(prompt):
    global last_update_id
    send_telegram_message(prompt)
    
    start_time = time.time()
    timeout = 300  # 5 minutes timeout

    while time.time() - start_time < timeout:
        updates = get_updates(last_update_id)
        for update in updates:
            last_update_id = update['update_id'] + 1
            if 'message' in update and 'text' in update['message']:
                return update['message']['text'].lower()
        time.sleep(1)
    
    send_telegram_message("No input received within 5 minutes. Using default value.")
    return None

def get_hyperparameters():
    choice = request_user_input("Do you want to input hyperparameters? (yes/no)")
    
    if choice == 'yes':
        learning_rate = float(request_user_input("Enter learning rate (e.g., 0.001):") or 0.001)
        batch_size = int(request_user_input("Enter batch size (e.g., 64):") or 64)
        num_epochs = int(request_user_input("Enter number of epochs (e.g., 2):") or 2)
        hidden_size = int(request_user_input("Enter hidden layer size (e.g., 512):") or 512)
    else:
        learning_rate = 0.001
        batch_size = 64
        num_epochs = 2
        hidden_size = 512
    
    send_telegram_message(f"Using hyperparameters: learning_rate={learning_rate}, batch_size={batch_size}, num_epochs={num_epochs}, hidden_size={hidden_size}")
    return learning_rate, batch_size, num_epochs, hidden_size

class SimpleNet(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

def train_model(num_epochs, train_loader, model, criterion, optimizer):
    model.train()
    epoch_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        epoch_loss /= len(train_loader)
        epoch_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
        send_telegram_message(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
    return epoch_losses

def check_file_sizes():
    max_file_size = 100 * 1024 * 1024  # 100 MB (GitHub's file size limit)
    acceptable_files = []
    for root, dirs, files in os.walk('.'):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.getsize(file_path) <= max_file_size:
                acceptable_files.append(file_path)
    return acceptable_files

def push_to_github(repo_url, is_private):
    try:
        # Initialize or open the repository
        if os.path.exists('.git'):
            repo = Repo('.')
        else:
            repo = Repo.init('.')
            repo.create_remote('origin', url=repo_url)

        # Stage all files
        repo.git.add(A=True)

        # Commit
        repo.index.commit("Update from Telegram bot")

        # Push
        origin = repo.remote('origin')
        origin.push()

        send_telegram_message("Successfully pushed to GitHub repository.")
    except Exception as e:
        send_telegram_message(f"Error pushing to GitHub: {str(e)}")

# Clear previous messages before starting the main loop
clear_previous_messages()

while True:
    learning_rate, batch_size, num_epochs, hidden_size = get_hyperparameters()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    model = SimpleNet(hidden_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    send_telegram_message("Model training has started.")
    epoch_losses = train_model(num_epochs, train_loader, model, criterion, optimizer)
    final_loss = epoch_losses[-1]
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.close()

    send_telegram_image('training_loss_plot.png')

    # GitHub integration
    use_github = request_user_input("Do you want to use GitHub? (yes/no)")
    if use_github == 'yes':
        acceptable_files = check_file_sizes()
        send_telegram_message(f"Files acceptable for GitHub push: {', '.join(acceptable_files)}")
        
        push_decision = request_user_input("Do you want to push these files to GitHub? (yes/no)")
        if push_decision == 'yes':
            repo_url = request_user_input("Enter your GitHub repository URL:")
            is_private = request_user_input("Is this a private repository? (yes/no)") == 'yes'
            push_to_github(repo_url, is_private)

    command = request_user_input("Enter 'rerun' to train again with new parameters, or 'stop' to end the program:")
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break
    elif command == "rerun":
        send_telegram_message("Rerunning the training with new parameters.")
        continue
    else:
        send_telegram_message("Invalid command. Stopping the program.")
        break

c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Cleared previous messages. Last update ID: 265526405
Sent message: Do you want to input hyperparameters? (yes/no), Response: {'ok': True, 'result': {'message_id': 248, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721912566, 'text': 'Do you want to input hyperparameters? (yes/no)'}}
Sent message: Using hyperparameters: learning_rate=0.001, batch_size=64, num_epochs=2, hidden_size=512, Response: {'ok': True, 'result': {'message_id': 250, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721912622, 'text': 'Using hyperparameters: learning_rate=0.001, batch_size=64, num_epochs=2, hidden_size=512'}}
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17307261.49it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 516636.22it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 7300526.96it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4517554.84it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Sent message: Model training has started., Response: {'ok': True, 'result': {'message_id': 251, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721912626, 'text': 'Model training has started.'}}
Epoch 1, Loss: 0.3107
Sent message: Epoch 1, Loss: 0.3107, Response: {'ok': True, 'result': {'message_id': 252, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721912677, 'text': 'Epoch 1, Loss: 0.3107'}}
Epoch 2, Loss: 0.1417
Sent message: Epoch 2, Loss: 0.1417, Response: {'ok': True, 'result': {'message_id': 253, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_nam